# 🌬️ Analyse av luftkvalitetsdata med NILU API
Denne notebooken dokumenterer hele arbeidsflyten for NILU-delen av prosjektet. Målet er å rense, transformere og visualisere luftkvalitetsdata (NO₂, O₃, SO₂) for bruk i senere analyse. 

### Datakilde: NILU API

**Kilde**: [api.nilu.no](https://api.nilu.no/)  
**Leverandør**: Norsk institutt for luftforskning (NILU)  
**Datakvalitet**: Høy – data fra offisielle målestasjoner  
**Tilgjengelighet**: Åpen (ingen API-nøkkel nødvendig for grunnleggende tilgang)  
**Brukervennlighet**: God, med REST-endepunkter og dokumentasjon

Valget av NILU API er basert på følgende kriterier:

- Kildeautoritet: NILU er en ledende institusjon innen luftkvalitetsforskning.
- Datakvalitet: Målinger fra sertifiserte og kalibrerte målestasjoner.
- Tilgjengelighet: Tilgjengelig for allmennheten uten behov for API-nøkkel.
- Brukervennlighet: Strukturert respons i JSON og fleksible parametere.
---

### ☁️ Henting og prosessering av luftkvalitetsdata (NILU API)

I denne delen av prosjektet benyttes **NILU API** for å hente luftkvalitetsdata for Oslo og nærliggende områder. Dataene struktureres og lagres i JSON-format.


#### Funksjonalitet

1. **Henting av luftkvalitetsdata**:
   - Geografisk område: Oslo
   - Koordinater: `59.9139`, `10.7522`
   - Radius: `20 km`
   - Periode: 2010-01-01 til 2016-04-02
   - Endepunkt: `https://api.nilu.no/stats/day/{from_date}/{to_date}/{latitude}/{longitude}/{radius}`

2. **Prosessering og lagring av data**:
   - Bearbeiding av JSON-respons
   - Strukturert dataeksport til:
     - Mappe: `../../data/raw_data/`
     - Filnavn: `raw_air_quality_nilu_oslo.json`

#### Viktige funksjoner

| Funksjon | Beskrivelse |
|---------|-------------|
| `get_air_quality_nilu_oslo()` | Setter opp og gjennomfører hele henteprosessen |
| `fetch_air_quality_nilu()` | Utfører selve API-kallet og lagrer strukturerte data som JSON |

#### Forutsetninger

- Python installert med nødvendige biblioteker: `requests`, `json`, `os`, `pathlib`
- Tilgang til [NILU API](https://api.nilu.no/)
- Skrivetilgang til katalogen: `../../data/raw_data/`


In [1]:
#Globale paramtere for niluAPI
import sys
import numpy as np
import pandas as pd
from pandasql import sqldf
import seaborn as sns
import matplotlib.pyplot as plt

base_url = "https://api.nilu.no/stats/day/"
from_date = "2010-04-02"
to_date = "2016-12-31"
radius=20

#Oslo kordinater
latitude= 59.9139
longitude=10.7522


In [2]:
sys.path.insert(0, '../../src')
from niluAPI.data_niluAPI import get_raw_data_niluAPI

get_raw_data_niluAPI()

Gruppert data er lagret under ../../data/raw_data/raw_air_quality_nilu_oslo.json


---
### 🧼 Rensing og klargjøring av NILU-data

Denne delen av prosjektet håndterer rensing og forbedring av luftkvalitetsdata hentet fra **NILU API**. Fokus ligger på å fjerne uteliggere, interpolere manglende data, og korrigere skjevheter i datafordelingen for å gjøre datasettet klarere og mer pålitelig for videre analyser.

#### Funksjonalitet

##### Fjerning av outliers og interpolering

- Leser rådata fra JSON-fil.
- Fjerner outliers på utvalgte gasser: `Verdi_NO2`, `Verdi_O3`, `Verdi_SO2`.
- Interpolerer manglende verdier for å opprettholde kontinuerlig dataserie.
- Lagrer renset data som JSON-fil.

##### Skjevhetskorreksjon av data

- Analyserer datasett for skjevhet i fordeling.
- Bruker Yeo-Johnson transformasjon for å normalfordele variablene.
- Lagrer korrigerte data for videre analyse.

#### Viktige funksjoner

##### `clean_raw_data()`

| Funksjon                | Beskrivelse                                                       |
|------------------------|------------------------------------------------------------------|
| **Laster inn rådata**    | Leser rå luftkvalitetsdata fra `raw_air_quality_nilu_oslo.json`. |
| **Fjerner outliers**     | Kaller `remove_outliers` for å identifisere og sette uteliggere til `NaN`. |
| **Interpolerer manglende verdier** | Bruker `interpolate_and_save_clean_data` for å fylle inn manglende data. |
| **Lagrer renset data**  | Lagres i `niluAPI_clean_data.json`.                              |

##### `fix_skewness_data_niluAPI()`

| Funksjon                | Beskrivelse                                                       |
|------------------------|------------------------------------------------------------------|
| **Analyserer skjevhet** | Bruker `analyse_and_fix_skewness` for å analysere fordelingens skjevhet i datasettet. |
| **Normalfordeler data** | Korrigerer skjevheter med Yeo-Johnson transformasjon.            |
| **Lagrer korrigerte data** | Resultatet lagres i `niluAPI_analyzed_data.json`.                |


#### Forutsetninger

- Rådata må være tilgjengelig i: `../../data/raw_data/raw_air_quality_nilu_oslo.json`
- Python-moduler fra prosjektmappen må være importert.
- Krever `sys` for path-håndtering og tilgang til spesialfunksjoner.
- Resultater lagres i:
  - Renset data: `../../data/clean_data/niluAPI_clean_data.json`
  - Analysert data: `../../data/analyzed_data/niluAPI_analyzed_data.json`


In [3]:
from niluAPI.data_niluAPI import clean_raw_data
clean_raw_data()

Fjerning av outliers:
Outliers er mer enn 3 standardavvik unna gjennomsnittet

Verdi_NO2:
Fjernet 39 outliers
Standaravvik: 21.74
Gjennomsnitt: 32.25

Verdi_O3:
Fjernet 8 outliers
Standaravvik: 16.29
Gjennomsnitt: 42.08

Verdi_SO2:
Fjernet 27 outliers
Standaravvik: 1.17
Gjennomsnitt: 2.23


Interpolering av Nan-verdier:
Verdi_NO2: 233 verdier ble interpolert
Verdi_O3: 574 verdier ble interpolert
Verdi_SO2: 393 verdier ble interpolert

Gruppert data er lagret under ../../data/clean_data/niluAPI_clean_data.json


In [4]:
from niluAPI.data_niluAPI import fix_skewness_data_niluAPI
fix_skewness_data_niluAPI()

Skjevhet før transformasjon:
→ Verdi_NO2: 1.12
→ Verdi_O3: -0.18
→ Verdi_SO2: 0.76

Behandler kolonner med skjevhet over ±1.0:

 Verdi_NO2: skjevhet 1.12 → Yeo-Johnson + skalering
Verdi_O3: skjevhet -0.18 → kun skalering
Verdi_SO2: skjevhet 0.76 → kun skalering

Skjevhet etter transformasjon:
→ Verdi_NO2_Trans: -0.01
→ Verdi_O3_Trans: -0.18
→ Verdi_SO2_Trans: 0.76

Transformert data lagret i: ../../data/analyzed_data/niluAPI_analyzed_data.json


---
## 📊  Analyse av luftkvalitetsdata fra NILU API (2010-2016)
I denne delen analyseres luftkvalitetsdata for Oslo over en seksårsperiode (2010-2016), hentet fra NILU API. Hensikten er å avdekke trender, sesongvariasjoner og potensielle forbedringer eller forverringer i luftkvaliteten over tid. Dataene inkluderer konsentrasjoner av NO₂, O₃ og SO₂, som er blant de viktigste indikatorene på lokal luftforurensning.

#### Metoder brukt for analyse:
- ...sett inn her
- ...sett inn her 

#### Analysestruktur:
- ...sett inn her
- ...sett inn her 

Alle analyser ledsages av visualiseringer, og resultatene tolkes med tanke på mulige miljø- og samfunnsmessige årsaker. Datasettet er renset og korrigert for skjevhet for å sikre robusthet i analysene.

In [5]:
#måntlig statistikk

df = pd.read_json("../../data/clean_data/niluAPI_clean_data.json")
query = """
SELECT 
    strftime('%Y-%m', Dato) AS Måned, 
    AVG(Verdi_NO2) AS Snitt_NO2,
    AVG(Verdi_O3) AS Snitt_O3,
    AVG(Verdi_SO2) AS Snitt_SO2,
    COUNT(*) AS AntallDager
FROM df
GROUP BY Måned
ORDER BY Måned
"""

    # Execute the query
pysqldf = lambda q: sqldf(q, globals())
monthly_stats = pysqldf(query)


print(monthly_stats)

      Måned  Snitt_NO2   Snitt_O3  Snitt_SO2  AntallDager
0   2010-04  35.294109  64.302514   2.862069           29
1   2010-05  21.199597  59.643414   2.394220           31
2   2010-06  20.604992  62.172153   1.661528           30
3   2010-07  19.245623  59.183333   1.759274           31
4   2010-08  22.120556  49.400958   1.744296           31
..      ...        ...        ...        ...          ...
76  2016-08  15.866144  31.959334   1.179835           31
77  2016-09  11.270938  29.060571   1.426891           30
78  2016-10  18.103540  30.474370   1.923178           31
79  2016-11  38.514975  25.146216   2.166528           30
80  2016-12  46.175656   8.485714   2.040703           31

[81 rows x 5 columns]


### 📊 Visualisering av luftkvalitet og datakvalitet

#### Hva gjør funksjonen?
Funksjonen `plot_air_quality()` genererer separate grafer for ulike luftforurensningskomponenter (NO₂, O₃, SO₂). Hver graf viser:

- En **punktsky** av måleverdier over tid.
- En **blå trendlinje** som indikerer utviklingen.
- **Fargekoding** av datapunkter basert på datakvalitet:
  - 🔴 **Rød** – manglende eller interpolerte verdier
  - 🟡 **Gul** – dekningsgrad under 90 %
  - 🟢 **Grønn** – god datakvalitet (≥ 90 %)

#### Fordeler ved denne fremstillingsmåten
- **Tydelig kommunikasjon av datakvalitet:** Brukeren får umiddelbar visuell informasjon om hvilke data som er pålitelige.
- **Støtter tolkning av trender:** Trendlinjer hjelper med å identifisere mønstre over tid.
- **Fleksibel og gjenbrukbar struktur:** Funksjonen tar generelle parametere og kan brukes på andre variabler, stasjoner og tidsperioder.
- **Styrker datadrevne beslutninger:** Fremhever både verdier og usikkerheter, noe som er avgjørende for miljøanalyse og -rapportering.



In [6]:
import sys

sys.path.insert(0, '../../src/') 
from niluAPI.data_niluAPI import load_and_plot_air_quality
load_and_plot_air_quality()

---
[⬅️ Forrige: Frost-analyse](../frostAPI/frostAPI.ipynb)                    
[➡️ Neste: Kombinert analyse](../Kombinert_analyse.ipynb) 